In [12]:

import os
import getpass
import requests
import time
from typing import Dict, Any, List
import pandas as pd

# 安全地注入 API Key（建议在 GCP 控制台为 Places API 启用后再粘贴）
if "GOOGLE_MAPS_API_KEY" not in os.environ:
    os.environ["GOOGLE_MAPS_API_KEY"] = getpass.getpass(
        "请输入你的 Google Maps API Key（不可见）: ")

API_KEY = os.environ["GOOGLE_MAPS_API_KEY"]
BASE = "https://places.googleapis.com/v1"


def call_places(method: str, path: str, field_mask: str,
                json_body: Dict[str, Any] | None = None,
                params: Dict[str, Any] | None = None) -> Dict[str, Any]:
    """通用调用器：封装 X-Goog-FieldMask / X-Goog-Api-Key 头"""
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": API_KEY,
        "X-Goog-FieldMask": field_mask,   # 必须！否则报错
    }
    url = f"{BASE}{path}"
    resp = requests.request(method, url, headers=headers,
                            json=json_body, params=params, timeout=30)
    resp.raise_for_status()
    return resp.json()

#1.Nearby Search（指定点与半径找“餐厅”）

• 仅支持 POST，请求体里写范围（圆形中心 + 半径）。

• 示例字段掩码只拿到 ID、地理位置、地址、评分、价格等用于做餐厅画像的关键字段

In [13]:
# 你可以换成自己的坐标与半径；这里示例：新加坡 Orchard Road 附近
center_lat, center_lng = 1.3049, 103.8318  # Orchard 附近
center_lat, center_lng = 45.553360, -122.674934  # 官方示例坐标（美国波特兰）
radius_m = 1500.0

# 为 Nearby 准备一个最小够用的字段掩码（不要包含 nextPageToken —— 官方页未明确 Nearby 的翻页字段）
SEARCH_MASK_NEARBY = ",".join([
    "places.id",
    "places.displayName",
    "places.location",
    "places.formattedAddress",
    # "places.primaryType",
    # "places.types",
    # "places.rating",
    # "places.userRatingCount",
    # "places.priceLevel",
    # "places.googleMapsUri",
    # "places.generativeSummary",
    # "places.editorialSummary"
])

nearby_body = {
    "includedTypes": ["restaurant"],  # 也可用 includedPrimaryTypes
    "maxResultCount": 20,             # Nearby 单页最多 20
    "rankPreference": "POPULARITY",   # 或 "DISTANCE"
    "locationRestriction": {
        "circle": {
            "center": {"latitude": center_lat, "longitude": center_lng},
            "radius": radius_m
        }
    }
}

nearby = call_places("POST", "/places:searchNearby",
                     SEARCH_MASK_NEARBY, json_body=nearby_body)
len(nearby.get("places", [])), nearby.get("places", [])[:5]  # 看下总数与前两个示例

(20,
 [{'id': 'ChIJOa08KlqnlVQR_ZZx1jEcTYY',
   'formattedAddress': '3808 N Williams Ave #127, Portland, OR 97227, USA',
   'location': {'latitude': 45.5506551, 'longitude': -122.66652119999998},
   'displayName': {'text': 'Eem - Thai BBQ & Cocktails',
    'languageCode': 'en'}},
  {'id': 'ChIJU4OzoWynlVQRxlQMpGenSvA',
   'formattedAddress': '4237 N Mississippi Ave, Portland, OR 97217, USA',
   'location': {'latitude': 45.554515699999996,
    'longitude': -122.67573579999998},
   'displayName': {'text': 'Prost!', 'languageCode': 'en'}},
  {'id': 'ChIJIxfmG0KnlVQR3ZPSgF46ahE',
   'formattedAddress': '3524 N Mississippi Ave, Portland, OR 97227, USA',
   'location': {'latitude': 45.5481556, 'longitude': -122.67532539999998},
   'displayName': {'text': '¿Por Qué No?', 'languageCode': 'en'}},
  {'id': 'ChIJeaQuoUeFhVQR5k_yPdK_gvk',
   'formattedAddress': '3925 NE Martin Luther King Jr Blvd, Portland, OR 97212, USA',
   'location': {'latitude': 45.551158099999995,
    'longitude': -122.66191

In [14]:
# 可视化
rows: List[Dict[str, Any]] = []
for p in nearby.get("places", []):
    # 注意：Nearby 返回的 location 可能是 {"latitude":..., "longitude":...}
    loc = p.get("location") or {}
    # 生成式摘要（Search 侧路径：places.generativeSummary）
    gsum = p.get("generativeSummary") or {}
    g_overview = ((gsum.get("overview") or {}).get(
        "text")) if isinstance(gsum, dict) else None
    # 编辑摘要
    esum = p.get("editorialSummary") or {}
    e_text = esum.get("text") if isinstance(esum, dict) else None

    rows.append({
        "id": p.get("id"),
        "name": (p.get("displayName") or {}).get("text"),
        "lat": loc.get("latitude"),
        "lng": loc.get("longitude"),
        "address": p.get("formattedAddress"),
        # "primaryType": p.get("primaryType"),
        # "types": ", ".join(p.get("types", [])),
        # "rating": p.get("rating"),
        # "ratingCount": p.get("userRatingCount"),
        # "priceLevel": p.get("priceLevel"),
        # "googleMapsUrl": p.get("googleMapsUri"),
        # 摘要拍平列
        # "genOverview": g_overview,
        # "editorialText": e_text,
    })

pd.DataFrame(rows)

,id,name,lat,lng,address
0,ChIJOa08KlqnlVQR_ZZx1jEcTYY,Eem - Thai BBQ & Cocktails,45.550655,-122.666521,"3808 N Williams Ave #127, Portland, OR 97227, USA"
1,ChIJU4OzoWynlVQRxlQMpGenSvA,Prost!,45.554516,-122.675736,"4237 N Mississippi Ave, Portland, OR 97217, USA"
2,ChIJIxfmG0KnlVQR3ZPSgF46ahE,¿Por Qué No?,45.548156,-122.675325,"3524 N Mississippi Ave, Portland, OR 97227, USA"
3,ChIJeaQuoUeFhVQR5k_yPdK_gvk,Cafe Olli,45.551158,-122.661911,"3925 NE Martin Luther King Jr Blvd, Portland, ..."
4,ChIJL9S2VWmnlVQRUhyawDjpzjY,The Alibi Tiki Lounge,45.552537,-122.680687,"4024 N Interstate Ave, Portland, OR 97227, USA"
5,ChIJT9ZbCWunlVQRIryTTXOWkEw,Lovely's Fifty Fifty,45.552860,-122.675850,"4039 N Mississippi Ave, Portland, OR 97217, USA"
6,ChIJb1BVEBenlVQRWQZoL6tdhpw,Matt's BBQ,45.554346,-122.675788,"4233 N Mississippi Ave, Portland, OR 97217, USA"
7,ChIJr8AyrWunlVQR_w411-qtqkw,Gravy,45.551708,-122.675706,"3957 N Mississippi Ave, Portland, OR 97227, USA"
8,ChIJ9SRcAkKnlVQRU2wR1eFxEew,Mississippi Pizza & Atlantis Lounge,45.548614,-122.675199,"3552 N Mississippi Ave, Portland, OR 97227, USA"
9,ChIJzQfQ8b2hlVQRz8RLZLvD_ps,Kate's Ice Cream,45.549804,-122.675777,"3713 N Mississippi Ave, Portland, OR 97227, USA"


#2.Text Search（关键词 + 区域偏置）

• POST 到 places:searchText，textQuery 可以写 “sushi near orchard road singapore”。

• pageSize 控制每页数量；响应里若返回 nextPageToken，用它继续翻页（放在请求体的 pageToken 字段）。


In [15]:
# Text Search 字段掩码：与 Nearby 类似，但加入 nextPageToken 以便分页
SEARCH_MASK_TEXT = ",".join([
    "places.id",
    "places.displayName",
    "places.location",
    "places.formattedAddress",
    "places.primaryType",
    "places.types",
    "places.rating",
    "places.userRatingCount",
    "places.priceLevel",
    "places.googleMapsUri",
    "places.generativeSummary",
    "nextPageToken"
])

text_body = {
    "textQuery": "sushi near orchard road singapore",
    # 用 locationBias 把结果“往某区域靠”
    "locationBias": {
        "circle": {
            "center": {"latitude": center_lat, "longitude": center_lng},
            "radius": 1200.0
        }
    },
    "pageSize": 10
}

page1 = call_places("POST", "/places:searchText",
                    SEARCH_MASK_TEXT, json_body=text_body)
print("第一页条数:", len(page1.get("places", [])))
print("是否可翻页:", "nextPageToken" in page1)

# 若可翻页，继续拿第二页
page2 = {}
if "nextPageToken" in page1:
    text_body2 = {**text_body, "pageToken": page1["nextPageToken"]}
    # 按官方示例，翻页前稍等一小会儿更稳妥
    time.sleep(1.2)
    page2 = call_places("POST", "/places:searchText",
                        SEARCH_MASK_TEXT, json_body=text_body2)
    print("第二页条数:", len(page2.get("places", [])))

第一页条数: 10
是否可翻页: True
第二页条数: 10


#3.Place Details（对单个餐厅补充细节）

• GET 到 /v1/places/{PLACE_ID}，同样必须带 FieldMask。
• 支持补充电话、网站、营业时间、评分/价位等（不同字段触发不同 SKU）

In [16]:
# 拿 Nearby 或 Text Search 的第一个结果做示例
any_place_id = (nearby.get("places") or page1.get(
    "places") or page2.get("places"))[0]["id"]

DETAILS_MASK = ",".join([
    # 注意：Place Details 的字段掩码是 Place 对象顶层，不再以 "places." 起头！
    "id",
    "displayName",
    "location",
    "formattedAddress",
    "primaryType",
    "types",
    "rating",
    "userRatingCount",
    "priceLevel",
    "websiteUri",
    "nationalPhoneNumber",
    "currentOpeningHours",
    "googleMapsUri",
    # 两类摘要
    "editorialSummary",
    "generativeSummary",
    # 一些“场景/服务”标签（可用于画像增强）
    "servesBreakfast",
    "servesBrunch",
    "servesLunch",
    "servesDinner",
    "servesVegetarianFood",
    "servesBeer",
    "servesWine",
    "servesCocktails",
])

details = call_places("GET", f"/places/{any_place_id}", DETAILS_MASK)
details

{'id': 'ChIJOa08KlqnlVQR_ZZx1jEcTYY',
 'types': ['thai_restaurant',
  'barbecue_restaurant',
  'bar',
  'restaurant',
  'food',
  'point_of_interest',
  'establishment'],
 'nationalPhoneNumber': '(971) 295-1645',
 'formattedAddress': '3808 N Williams Ave #127, Portland, OR 97227, USA',
 'location': {'latitude': 45.5506551, 'longitude': -122.66652119999998},
 'rating': 4.7,
 'googleMapsUri': 'https://maps.google.com/?cid=9677422174665807613&g_mp=CiVnb29nbGUubWFwcy5wbGFjZXMudjEuUGxhY2VzLkdldFBsYWNlEAAYBCAA',
 'websiteUri': 'http://www.eempdx.com/',
 'priceLevel': 'PRICE_LEVEL_MODERATE',
 'userRatingCount': 2527,
 'displayName': {'text': 'Eem - Thai BBQ & Cocktails', 'languageCode': 'en'},
 'servesBreakfast': False,
 'servesLunch': True,
 'servesDinner': True,
 'servesBeer': True,
 'servesWine': True,
 'servesBrunch': False,
 'servesVegetarianFood': True,
 'currentOpeningHours': {'openNow': False,
  'periods': [{'open': {'day': 0,
     'hour': 11,
     'minute': 0,
     'date': {'year': 2

In [17]:
loc = p.get("location") or {}

gsum = details.get("generativeSummary") or {}
g_overview = ((gsum.get("overview") or {}).get(
    "text")) if isinstance(gsum, dict) else None

esum = details.get("editorialSummary") or {}
e_text = esum.get("text") if isinstance(esum, dict) else None


row = {
    "id": details.get("id"),
    "name": (details.get("displayName") or {}).get("text"),
    "lat": loc.get("latitude"),
    "lng": loc.get("longitude"),
    "address": details.get("formattedAddress"),
    "primaryType": details.get("primaryType"),
    "types": ", ".join(details.get("types", [])),
    "rating": details.get("rating"),
    "ratingCount": details.get("userRatingCount"),
    "priceLevel": details.get("priceLevel"),
    "googleMapsUrl": details.get("googleMapsUri"),
    "servesBreakfast": details.get("servesBreakfast"),
    "servesBrunch": details.get("servesBrunch"),
    "servesLunch": details.get("servesLunch"),
    "servesDinner": details.get("servesDinner"),
    "servesVegetarianFood": details.get("servesVegetarianFood"),
    "servesBeer": details.get("servesBeer"),
    "servesWine": details.get("servesWine"),
    "servesCocktails": details.get("servesCocktails"),
    "website": details.get("websiteUri"),
    "phone": details.get("nationalPhoneNumber"),
    # 摘要拍平列
    "gen_overview": g_overview,
    "editorial_text": e_text,
}

pd.DataFrame([row])

,id,name,lat,lng,address,primaryType,types,rating,ratingCount,priceLevel,...,servesLunch,servesDinner,servesVegetarianFood,servesBeer,servesWine,servesCocktails,website,phone,gen_overview,editorial_text
0,ChIJOa08KlqnlVQR_ZZx1jEcTYY,Eem - Thai BBQ & Cocktails,45.55197,-122.675274,"3808 N Williams Ave #127, Portland, OR 97227, USA",thai_restaurant,"thai_restaurant, barbecue_restaurant, bar, res...",4.7,2527,PRICE_LEVEL_MODERATE,...,True,True,True,True,True,True,http://www.eempdx.com/,(971) 295-1645,"BBQ and Thai street fare, plus imaginative tro...",Modern Thai eatery serving creative BBQ and cr...


In [18]:
GEOLOCATION_URL = "https://www.googleapis.com/geolocation/v1/geolocate"


def get_current_location_via_geolocation_api(consider_ip: bool = True):
    """
    使用 Google Geolocation API 估算当前位置。
    - 仅传 considerIp 时：精度≈城市级（依赖 IP）
    - 若你在移动端/前端收集到 wifiAccessPoints/cellTowers，可附带提交以提升精度
      （Colab/后端通常拿不到这些数据）
    """
    params = {"key": API_KEY}
    payload = {"considerIp": consider_ip}
    r = requests.post(GEOLOCATION_URL, params=params, json=payload, timeout=15)
    r.raise_for_status()
    data = r.json()
    # 返回形如 {"location":{"lat":..,"lng":..}, "accuracy":..}
    loc = data.get("location", {})
    return {"lat": loc.get("lat"), "lng": loc.get("lng"), "accuracy_m": data.get("accuracy")}


location = get_current_location_via_geolocation_api(consider_ip=True)
location

{'lat': 1.3336576, 'lng': 103.8516224, 'accuracy_m': 12110.827729521425}